In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
    library(pROC)
}))


In [2]:
# input

phe_f <- '/oak/stanford/groups/mrivas/projects/biomarkers/snpnet/disease_outcome/phe.tsv'
PRS_f <- '/oak/stanford/groups/mrivas/projects/biomarkers/snpnet/disease_outcome/HC294/results/score/HC294.sscore'


In [3]:
phe <- fread(
    phe_f,
    select=c('FID', 'IID', 'split', 'HC294'), 
    colClasses=c('FID'='character', 'IID'='character', 'split'='character')
)

PRS <- fread(
    PRS_f, 
    select=c('#FID', 'IID', 'SCORE1_SUM'), 
    colClasses=c('#FID'='character', 'IID'='character')
) %>%
rename('FID'='#FID')

phe %>%
filter(split != '') %>%
left_join(
    PRS %>% rename('PRS' = 'SCORE1_SUM'), by=c('FID', 'IID')
) %>%
mutate(HC294 = HC294 - 1) -> phe_PRS


In [4]:
phe_PRS %>% filter(split == 'test') -> phe_PRS_test


In [5]:
roc_res <- roc(HC294 ~ PRS, phe_PRS_test, direction=c("<"), levels=c('control' = 0, 'case' = 1))


In [6]:
roc_res


Call:
roc.formula(formula = HC294 ~ PRS, data = phe_PRS_test, direction = c("<"),     levels = c(control = 0, case = 1))

Data: PRS in 66005 controls (HC294 0) < 1425 cases (HC294 1).
Area under the curve: 0.4953

In [7]:
ci(roc_res)

95% CI: 0.4801-0.5105 (DeLong)

In [8]:
wilcox.test(
    phe_PRS_test %>% filter(HC294 == 0) %>% pull(PRS),
    phe_PRS_test %>% filter(HC294 == 1) %>% pull(PRS)
)



	Wilcoxon rank sum test with continuity correction

data:  phe_PRS_test %>% filter(HC294 == 0) %>% pull(PRS) and phe_PRS_test %>% filter(HC294 == 1) %>% pull(PRS)
W = 47471334, p-value = 0.5425
alternative hypothesis: true location shift is not equal to 0


https://github.com/xrobin/pROC/wiki/FAQ---Frequently-asked-questions#can-i-test-if-a-single-roc-curve-is-significantly-different-from-05


In [10]:
?ci

ci {pROC},R Documentation
"roc, smooth.roc","a “roc” object from the roc function, or a “smooth.roc” object from the smooth function."
"multiclass.roc, multiclass.auc",not implemented.
"response, predictor",arguments for the roc function.
"formula, data",a formula (and possibly a data object) of type response~predictor for the roc function.
of,"The type of confidence interval. One of “auc”, “thresholds”, “sp”, “se” or “coords”. Note that confidence interval on “thresholds” are not available for smoothed ROC curves."
...,"further arguments passed to or from other methods, especially auc, roc, and the specific ci functions ci.auc, ci.se, ci.sp and ci.thresholds."
